# 시뮬레이션 환경: Gymnasium

LLM(대형 언어 모델) 에이전트의 많은 응용 프로그램에서는 환경이 실제(인터넷, 데이터베이스, REPL 등)이다. 그러나 텍스트 기반 게임과 같은 시뮬레이션된 환경에서 상호작용할 수 있는 에이전트를 정의할 수도 있다. 이 예제는 [Gymnasium](https://github.com/Farama-Foundation/Gymnasium) (이전의 [OpenAI Gym](https://github.com/openai/gym))을 사용하여 간단한 에이전트-환경 상호작용 루프를 만드는 방법을 보여준다.

In [1]:
!pip install gymnasium

In [2]:
import tenacity
from langchain.output_parsers import RegexParser
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

## 에이전트 정의

In [3]:
class GymnasiumAgent:
    @classmethod
    def get_docs(cls, env):
        return env.unwrapped.__doc__

    def __init__(self, model, env):
        self.model = model
        self.env = env
        self.docs = self.get_docs(env)

        self.instructions = """
당신의 목표는 당신이 받는 보상의 합계를 최대화하는 것이다.
관찰, 보상, 종료 플래그, 자르기 플래그, 그리고 지금까지의 보상의 합계를 다음과 같은 형식으로 제공하겠다:

관찰: <observation>
보상: <reward>
종료: <termination>
자르기: <truncation>
합계: <sum_of_rewards>

당신은 다음과 같은 형식으로 행동을 응답해야 한다:

행동: <action>

여기서 <action>을 실제 행동으로 대체해야 한다.
다른 것은 하지 말고 행동만 반환해야 한다.
"""
        self.action_parser = RegexParser(
            regex=r"Action: (.*)", output_keys=["action"], default_output_key="action"
        )

        self.message_history = []
        self.ret = 0

    def random_action(self):
        action = self.env.action_space.sample()
        return action

    def reset(self):
        self.message_history = [
            SystemMessage(content=self.docs),
            SystemMessage(content=self.instructions),
        ]

    def observe(self, obs, rew=0, term=False, trunc=False, info=None):
        self.ret += rew

        obs_message = f"""
관찰: {obs}
보상: {rew}
종료: {term}
자르기: {trunc}
합계: {self.ret}
        """
        self.message_history.append(HumanMessage(content=obs_message))
        return obs_message

    def _act(self):
        act_message = self.model.invoke(self.message_history)
        self.message_history.append(act_message)
        action = int(self.action_parser.parse(act_message.content)["action"])
        return action

    def act(self):
        try:
            for attempt in tenacity.Retrying(
                stop=tenacity.stop_after_attempt(2),
                wait=tenacity.wait_none(),  # 재시도 간 대기 시간 없음
                retry=tenacity.retry_if_exception_type(ValueError),
                before_sleep=lambda retry_state: print(
                    f"ValueError 발생: {retry_state.outcome.exception()}, 재시도 중..."
                ),
            ):
                with attempt:
                    action = self._act()
        except tenacity.RetryError:
            action = self.random_action()
        return action

## 시뮬레이션 환경 및 에이전트 초기화

In [4]:
env = gym.make("Blackjack-v1")
agent = GymnasiumAgent(model=ChatOpenAI(temperature=0.2), env=env)

## 메인 루프

In [5]:
observation, info = env.reset()
agent.reset()

obs_message = agent.observe(observation)
print(obs_message)

while True:
    action = agent.act()
    observation, reward, termination, truncation, info = env.step(action)
    obs_message = agent.observe(observation, reward, termination, truncation, info)
    print(f"행동: {action}")
    print(obs_message)

    if termination or truncation:
        print("break", termination, truncation)
        break
env.close()


관찰: (15, 4, 0)
보상: 0
종료: False
자르기: False
합계: 0
        
행동: 1

관찰: (25, 4, 0)
보상: -1.0
종료: True
자르기: False
합계: -1.0
        
break True False
